In [1]:
import numpy as np
import pandas as pd

from scipy.optimize import minimize

In [62]:
date_range = pd.date_range(start='2022-01-01', end='2023-12-31', freq='D')
saisies = pd.Series(np.random.randint(1500, 3001, size=len(date_range)), index=date_range)
initial_receptions = saisies.apply(lambda x: x * np.random.uniform(0.2, 1.2))


Approche	Avantages	Inconvénients
Constrained Optimization	Contraintes directement intégrées dans le modèle, résultats optimisés globalement.	Plus complexe à implémenter, nécessite de bien calibrer les pénalités.
Post-processing	Simple à mettre en œuvre, adaptable à tout modèle préexistant.	Résolution a posteriori, peut réduire la précision des prévisions.
Régularisation avec XGBoost	Modèles performants et scalables, facile à intégrer dans un pipeline existant.	Fonction objective plus complexe à implémenter.

# Custom objective pour LightGBM

In [ ]:
import numpy as np
import lightgbm as lgb

def custom_objective(y_true, y_pred, saisies_pred, lambda_reg=1.0):
    grad = 2 * (y_pred - y_true)
    hess = 2 * np.ones_like(y_pred)
    # Pénalité si réceptions > saisies
    penalty = np.clip(y_pred - saisies_pred, 0, None)
    grad += lambda_reg * penalty
    return grad, hess

lgb.Dataset(X_train, label=y_train)
params = {
    'objective': 'regression',
    'metric': 'mse'
}
model = lgb.train(params, train_data, fobj=lambda y_true, y_pred: custom_objective(y_true, y_pred, saisies_pred))

# Pendant l'entrainement

Définition d'une fonction loss custom qu'on va venir passer à un modèle comme XGBoost (custom objective)

In [ ]:
import torch

def custom_loss(y_true, y_pred, saisies_pred, lambda_reg=1.0):
    mse = torch.mean((y_true - y_pred) ** 2)
    constraint_violation = torch.mean(torch.clamp(y_pred - saisies_pred, min=0))  # Pénalité si réception > saisie
    return mse + lambda_reg * constraint_violation

# Post traitement

In [43]:
def objective(receptions):
    return np.sum((receptions - initial_receptions)**2)

def constraint_moving_avg(receptions):
    rolling_saisies = saisies.rolling(window=30).mean().dropna()
    rolling_receptions = pd.Series(receptions).rolling(window=30).mean().dropna()
    return rolling_saisies - rolling_receptions

In [ ]:
constraints = {"type": "ineq", "fun": constraint_moving_avg}

constraints_2 = [
    {'type': 'ineq', 'fun': lambda y_adj: saisies - y_adj},  # y_adj <= saisies
    {'type': 'ineq', 'fun': lambda y_adj: np.sum(saisies) - np.sum(y_adj)}  # somme(y_adj) <= somme(saisies)
]

bounds = [(0, None) for _ in y_init]

result = minimize(objective, initial_receptions, constraints=constraints, method="SLSQP")

optimized_receptions = pd.Series(result.x, index=date_range)

In [68]:
df = pd.DataFrame({
    "saisies": saisies,
    "receptions_initiales": initial_receptions,
    "receptions_optimises": optimized_receptions
})

df.head()

,saisies,receptions_initiales,receptions_optimises
2022-01-01,1861,847.875036,847.875036
2022-01-02,1939,646.033646,646.033646
2022-01-03,2659,1578.799398,1578.799398
2022-01-04,2361,1503.000126,1503.000126
2022-01-05,2686,884.425987,884.425987
